In [ ]:
# # Seleção do que desejamos
    # listsofA = soup.select(".hoverinfo_trigger a")
    
    # # Loop para extrair o atributo href da tag a
    # for a in listsofA:
    #     links.append(a.get("href"))
        
    # # Abre o arquivo em disco
    # file = open(f"dados/AnimeLinks/animeLinks{limit}.txt", "w")
    
    # # Loop para salvar em disco
    # for link in links:
    #     file.writelines(link+'\n')
        
    # # Fecha o arquivo
    # file.close()
    
    # # Sleep
    # time.sleep(10)

In [2]:
# Imports
import re
import bs4
import time
import json
import requests
from bs4 import BeautifulSoup as bs

In [113]:
# URL
search = "https://www.linkedin.com/jobs/search/?currentJobId=3743834990&keywords=data%20analyst&origin=SWITCH_SEARCH_VERTICAL"

jobs_list = []

def getJobsURL(url_search: str , start_page: int = 0, num_pages: int = 1):
        
    for page in range(0, num_pages):

        job_page = num_pages * 25

        url = url_search + f'&start={job_page}'

        # Request
        rq = requests.get(url)
        print(rq.status_code)

        #Parse do html
        soup = bs(rq.text, "html.parser")

        list_href = soup.select('.base-card__full-link')

        for i in range(len(list_href)):
            jobs_list.append(list_href[i]['href'])

    return jobs_list
    
jobs_list = getJobsURL(search)
#print(soup.prettify())   

200


In [ ]:
# Iterate over the jobsURL
soup_jobs_list = []
def getJobsInfo(url_search: str , start_page: int = 0, num_pages: int = 1):

    jobs_list = getJobsURL(url_search = search, start_page = start_page, num_pages = num_pages)

    for job in range(len(jobs_list)): 
        url = jobs_list[job]

        # Request
        rq = requests.get(url)
        print(rq.status_code)

        #Parse do html
        soup_job = bs(rq.text, "html.parser")

        soup_jobs_list.append(soup_job)
    
    return soup_jobs_list

soup_list = getJobsInfo(search)
soup_list

In [87]:
url = jobs_list[2]

# Request
rq = requests.get(url)
#print(rq.status_code)

#print(rq.text)
#Parse do html
soup_job = bs(rq.text, "html.parser")

In [121]:
soup_list[0].find('script', attrs = {'type':'application/ld+json'}).text

'\n      {"@context":"http://schema.org","@type":"JobPosting","datePosted":"2023-10-31T00:00:00.000Z","description":"&lt;strong&gt;What We Strive For&lt;br&gt;&lt;br&gt;&lt;/strong&gt;Strive Health is built for purpose — to transform a broken kidney care system. We are fundamentally changing the lives of kidney disease patients through early identification, engagement and comprehensive coordinated care. Strive’s model is driven by a high-touch care team that integrates with local providers and spans the entire care journey from chronic kidney disease through end-stage kidney disease, leveraging comparative and predictive data and analytics to identify at-risk patients. Strive Health’s interventions significantly reduce the rate of emergent dialysis crash, cut inpatient utilization and significantly improve patient outcomes and experience. Come join our journey as we create THE destination for top talent in the healthcare community and set a new standard for how kidney care should be do

In [160]:
import json

def getJobAttr(soup_job):

    # String JSON
    json_string = soup_job.find_all('script', attrs = {'type':'application/ld+json'})[0].text

    # Convert a string JSON to Python Object
    json_obj = json.loads(json_string)

    # Now, the json is a Python dictionary
    return json_obj

getJobAttr(soup_list[0])

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2023-10-31T00:00:00.000Z',
 'description': '&lt;strong&gt;What We Strive For&lt;br&gt;&lt;br&gt;&lt;/strong&gt;Strive Health is built for purpose — to transform a broken kidney care system. We are fundamentally changing the lives of kidney disease patients through early identification, engagement and comprehensive coordinated care. Strive’s model is driven by a high-touch care team that integrates with local providers and spans the entire care journey from chronic kidney disease through end-stage kidney disease, leveraging comparative and predictive data and analytics to identify at-risk patients. Strive Health’s interventions significantly reduce the rate of emergent dialysis crash, cut inpatient utilization and significantly improve patient outcomes and experience. Come join our journey as we create THE destination for top talent in the healthcare community and set a new standard for how kidney care should be d

In [124]:
#### Metadados
def getJobMetaData(soup_job):
    postMetaData = {}

    postMetaData['jobURL'] = soup_job.find_all('meta', attrs = {'property':'og:url'})[0]['content']
    postMetaData['jobTitle'] = soup_job.find('title').text
    postMetaData['jobLocale'] = soup_job.find_all('meta', attrs = {'name':'locale'})[0]['content']
    postMetaData['jobTimePosted'] = ((soup_job.find_all('meta', attrs = {'name':'description'})[0]['content']).split('.')[0]).replace('Posted ', '')
    postMetaData['jobCompanyId'] = soup_job.find_all('meta', attrs = {'name':'companyId'})[0]['content']
    postMetaData['jobIndustryId'] = soup_job.find_all('meta', attrs = {'name':'industryIds'})[0]['content']
    # postMetaData['jobDescription'] = soup_job.find_all('meta', attrs = {'name':'description'})[0]['content']

    return postMetaData

getJobMetaData(soup_list[0])

{'jobURL': 'https://www.linkedin.com/jobs/view/data-analyst-partner-analytics-at-strive-health-3751182198',
 'jobTitle': 'Strive Health hiring Data Analyst, Partner Analytics in Denver, CO | LinkedIn',
 'jobLocale': 'en_US',
 'jobTimePosted': '12:00:00 AM',
 'jobCompanyId': '33237712',
 'jobIndustryId': '14'}

In [158]:
def getFullJobDescription(soup_job):

    boldItens = []

    for item in range(len(soup_job.select('.show-more-less-html__markup strong'))):
        boldItens.append(soup_job.select('.show-more-less-html__markup strong')[item].text)

    jobDescription = (soup_job.select('.show-more-less-html__markup')[0].text).replace('\n', ' ')

    fullDescription = {
                        'boldItens': boldItens,
                        'jobDescription': jobDescription
                        }
                        
    return fullDescription

getFullJobDescription(soup_list[0])

{'boldItens': ['What We Strive For',
  'Why We’re Worth the Application',
  'Data Analyst (Partner Analytics)',
  'Essential Functions',
  'Minimum Qualifications',
  'Preferred Qualifications',
  'Annual Salary Range:'],
 'jobDescription': ' What We Strive ForStrive Health is built for purpose — to transform a broken kidney care system. We are fundamentally changing the lives of kidney disease patients through early identification, engagement and comprehensive coordinated care. Strive’s model is driven by a high-touch care team that integrates with local providers and spans the entire care journey from chronic kidney disease through end-stage kidney disease, leveraging comparative and predictive data and analytics to identify at-risk patients. Strive Health’s interventions significantly reduce the rate of emergent dialysis crash, cut inpatient utilization and significantly improve patient outcomes and experience. Come join our journey as we create THE destination for top talent in the

In [161]:
def getJobPost(soup_job):
    jobPost = {
        'attributes': getJobAttr(soup_job),
        'metadata': getJobMetaData(soup_job),
        'fullDescription': getFullJobDescription(soup_job)
    }
    return jobPost

getJobPost(soup_list[0])

{'attributes': {'@context': 'http://schema.org',
  '@type': 'JobPosting',
  'datePosted': '2023-10-31T00:00:00.000Z',
  'description': '&lt;strong&gt;What We Strive For&lt;br&gt;&lt;br&gt;&lt;/strong&gt;Strive Health is built for purpose — to transform a broken kidney care system. We are fundamentally changing the lives of kidney disease patients through early identification, engagement and comprehensive coordinated care. Strive’s model is driven by a high-touch care team that integrates with local providers and spans the entire care journey from chronic kidney disease through end-stage kidney disease, leveraging comparative and predictive data and analytics to identify at-risk patients. Strive Health’s interventions significantly reduce the rate of emergent dialysis crash, cut inpatient utilization and significantly improve patient outcomes and experience. Come join our journey as we create THE destination for top talent in the healthcare community and set a new standard for how kidne

In [15]:
soup_jobs_topics = soup_job.select('.show-more-less-html__markup strong')
# [<strong>About Nevro<br/><br/></strong>,
#  <strong>Job Summary &amp; Responsibilities<br/><br/></strong>,
#  <strong>Essential Duties And Responsibilities<br/><br/></strong>,
#  <strong>Role Requirements<br/><br/></strong>,
#  <strong>Skills And Knowledge<br/><br/></strong>,
#  <strong> Target Pay Range <br/><br/></strong>,
#  <strong> EEO Statement <br/><br/></strong>]

# topics_jobs = soup_job.select('.show-more-less-html strong')
# # [<strong>About Nevro<br/><br/></strong>,
# #  <strong>Job Summary &amp; Responsibilities<br/><br/></strong>,
# #  <strong>Essential Duties And Responsibilities<br/><br/></strong>,
# #  <strong>Role Requirements<br/><br/></strong>,
# #  <strong>Skills And Knowledge<br/><br/></strong>,
# #  <strong> Target Pay Range <br/><br/></strong>,
# #  <strong> EEO Statement <br/><br/></strong>]

list_jobs_topics = []

print('-------- tópicos do job-----------')
for i in range(len(soup_jobs_topics)):
    if soup_jobs_topics[i].text != '':
        list_jobs_topics.append(soup_jobs_topics[i].text)

print(list_jobs_topics)
print('-----------------------------------')

print('-------- descrição do job-----------')
soup_job_description = soup_job.select('.show-more-less-html__markup')[0].contents

# ['\n',
#  <strong>About Nevro<br/><br/></strong>,
#  'Nevro (NYSE: NVRO) is a global medical device company headquartered in Redwood City, California. We are focused on delivering comprehensive, life-changing solutions that continue to set the standard for enduring patient outcomes in chronic pain treatment. The company started with a simple mission to help more patients suffering from debilitating pain and developed its proprietary 10 kHz Therapy™, an evidence-based, non-pharmacologic innovation that has impacted the lives of more than 100,000 patients globally.',
#  <br/>,
#  <br/>,
#  "Nevro’s comprehensive HFX™ spinal cord stimulation (SCS) platform includes a Senza SCS system and support services for the treatment of chronic trunk and limb pain and painful diabetic neuropathy. Senza®, Senza II®, Senza Omnia™, and Senza HFX iQ™ are the only SCS systems that deliver Nevro's proprietary 10 kHz Therapy. HFX iQ, Nevro’s latest innovation, is the first and only SCS System that uses Artificial Intelligence to optimize and maintain pain relief using each patient's response 1 .Nevro’s unique support services provide every patient with an HFX Coach™ throughout their pain relief journey and every physician with HFX Cloud™ insights for enhanced patient and practice management.",
#  <br/>,
#  <br/>,
#  <strong>Job Summary &amp; Responsibilities<br/><br/></strong>,
#  'Nevro is seeking a Data Analyst who will be responsible for turning data into information, to provide insight into data trends to assist in making business decisions. This role will also support various ad-hoc projects.',
#  <br/>,
#  .........

test = {}

print('dictionary')
c = 0


for i in range(len(soup_job_description)):
    if soup_job_description[i].text in list_jobs_topics:
       # list_topics.append(soup_job_description[i].text)
        currently_list = str(soup_job_description[i].text)
        test[str(currently_list)] = []
        c = 1
    
    elif c != 0:
        if soup_job_description[i].text not in list_jobs_topics:
            if soup_job_description[i].text not in ['', '\n']:
                test[str(currently_list)].append(soup_job_description[i].text)

print(test)
print(soup_job_description)
# topics_jobs = soup_job.select('.show-more-less-html strong')

-------- tópicos do job-----------
['Job Description', 'FOIA Jr. Data AnalystEssential Job Functions: ', 'Skills:', 'Qualifications:', 'IFAS is an Equal Opportunity/Affirmative Action employer. All qualified applicants will receive consideration for employment without regard to race, color, religion, sex, sexual orientation, gender identity, national origin, disability, or veteran status, or any other protected class.', 'Company Description']
-----------------------------------
-------- descrição do job-----------
dictionary
{'Job Description': [], 'FOIA Jr. Data AnalystEssential Job Functions: ': ['The FOIA Junior Data Analyst scope entails supporting our clients with onsite records analysis and administrative support for its Freedom of Information Act (FOIA) operations. Our client receives a large number of FOIA/Privacy act requests annually. We need administrative support to facilitate the reduction of the FOIA and Privacy Act (PA) requests and referrals backlog and processing of cu

In [125]:
soup_list[0].select('.show-more-less-html__markup')[0].contents.

['\n',
 <strong>What We Strive For<br/><br/></strong>,
 'Strive Health is built for purpose — to transform a broken kidney care system. We are fundamentally changing the lives of kidney disease patients through early identification, engagement and comprehensive coordinated care. Strive’s model is driven by a high-touch care team that integrates with local providers and spans the entire care journey from chronic kidney disease through end-stage kidney disease, leveraging comparative and predictive data and analytics to identify at-risk patients. Strive Health’s interventions significantly reduce the rate of emergent dialysis crash, cut inpatient utilization and significantly improve patient outcomes and experience. Come join our journey as we create THE destination for top talent in the healthcare community and set a new standard for how kidney care should be done.',
 <br/>,
 <br/>,
 <strong>Why We’re Worth the Application</strong>,
 ' ',
 <br/>,
 <br/>,
 <ul><li>We Strive for excellenc